In [86]:
import pandas as pd
import numpy as np
import os
import forzawrapper

<h1>Gruppspel</h1>

In [87]:
def get_sign(score):
    if type(score) != str:
        return None
    h, a = int(score[0]), int(score[2])
    if h == a:
        return "X"
    elif h > a:
        return "1"
    else:
        return "2"

files = ["sheets/"+f for f in os.listdir("sheets")]

names = []
preds = []
for f in files:
    names.append('-'.join(f.split("-")[2:]).split(".")[0])
    tmp = pd.read_excel(f)
    preds.append(tmp[tmp.columns[5]][11:59].values.astype(str))

forza_df = forzawrapper.get_scores().iloc[:len(preds[0])].drop("Kickoff", axis=1)

pred_df = pd.DataFrame(np.array(preds).T, columns=names)
df = pd.concat([pred_df, forza_df], axis=1)
df["Correct"] = df["Score"].apply(lambda x: get_sign(x))
df["Correct Predictions"] = df.apply(lambda x: sum([int(v == x["Correct"]) for v in x[df.columns[:4]]]), axis=1)
df["Points Given"] = 0.5*(1+(len(files)*2-df["Correct Predictions"]))*df["Correct Predictions"].astype(bool)
df

,Irannejad-Armin,Filip,Martin-Hunhammar,Sellberg,Teams,Score,Correct,Correct Predictions,Points Given
0,2,2,X,2,Qatar vs Ecuador,0-2,2,3,3.0
1,X,1,1,1,England vs Iran,6-2,1,3,3.0
2,2,2,2,2,Senegal vs Netherlands,0-2,2,4,2.5
3,1,X,2,2,USA vs Wales,1-1,X,1,4.0
4,1,1,1,1,Argentina vs Saudi Arabia,1-2,2,0,0.0
5,1,1,1,1,Denmark vs Tunisia,0-0,X,0,0.0
6,1,2,X,2,Mexico vs Poland,0-0,X,1,4.0
7,1,1,1,1,France vs Australia,4-1,1,4,2.5
8,2,1,2,2,Morocco vs Croatia,0-0,X,0,0.0
9,1,1,1,1,Germany vs Japan,1-2,2,0,0.0


In [91]:
print("Antal gruppspelspoäng efter", (~df['Correct'].isna()).sum(), "matcher")
names = []
scores = []
percent = []
points = []
for c in df.columns[:4]:
    names.append(c)
    scores.append((df[c] == df['Correct']).sum())
    percent.append(str(((df[c] == df['Correct']).sum()/(~df['Correct'].isna()).sum())*100)+"%")
    points.append(((df[c] == df['Correct']).astype(int)*df["Points Given"]).sum())
    
table = pd.DataFrame([names, points, scores, percent]).T
table.columns = ["Namn", "Poäng", "Antal Rätt", "Procent"]
table.sort_values("Poäng", ascending=False).reset_index(drop=True)

Antal gruppspelspoäng efter 25 matcher


,Namn,Poäng,Antal Rätt,Procent
0,Martin-Hunhammar,42.5,14,56.00000000000001%
1,Filip,33.5,12,48.0%
2,Sellberg,33.5,12,48.0%
3,Irannejad-Armin,24.5,9,36.0%


<h1>Slutspel</h1>

In [92]:
country_map = {x[1]:x[0] for x in pd.read_html("https://svenskaengelska.se/lander.html")[0].values}
qual_preds = []
for f in files:
    tmp = pd.read_excel(f)
    qual_preds.append([t.split(" ")[0] for t in tmp[tmp.columns[5]][61:77].values])
qual_df = pd.DataFrame(qual_preds).T
qual_df.columns = names
qual_df["Correct"] = forzawrapper.get_qualifying()
qual_df["Correct"] = qual_df["Correct"].apply(lambda x: country_map[x] if x in country_map.keys() else x)
qual_df["Correct Predictions"] = qual_df["Correct"].apply(lambda x: sum([int(x in qual_df[c].values) for c in qual_df.columns[:-1]]))
qual_df["Points Given"] = 0.5*(1+(len(files)*2-qual_df["Correct Predictions"]))*qual_df["Correct Predictions"].astype(bool)
qual_df

,Irannejad-Armin,Filip,Martin-Hunhammar,Sellberg,Correct,Correct Predictions,Points Given
0,Nederländerna,Nederländerna,Nederländerna,Nederländerna,Nederländerna,4,2.5
1,Senegal,Senegal,Senegal,Ecuador,Ecuador,1,4.0
2,England,England,England,England,England,4,2.5
3,Iran,Wales,Wales,Wales,Iran,1,4.0
4,Argentina,Argentina,Argentina,Argentina,Polen,3,3.0
5,Mexiko,Polen,Polen,Polen,Argentina,4,2.5
6,Frankrike,Danmark,Frankrike,Frankrike,Frankrike,4,2.5
7,Danmark,Frankrike,Danmark,Danmark,Australien,0,0.0
8,Tyskland,Spanien,Tyskland,Spanien,Spanien,4,2.5
9,Spanien,Tyskland,Spanien,Tyskland,Japan,0,0.0


In [93]:
print("Antal poäng från lag kvalificerade till slutspel, utifrån gruppställning efter", (~df['Correct'].isna()).sum(), "matcher")

qual_score_df = pd.DataFrame(list(qual_df.apply(lambda x: [x["Points Given"]*int(x["Correct"] in qual_df[c].values) for c in qual_df.columns[:4]], axis=1).values), columns=qual_df.columns[:4]).sum().to_frame()
qual_score_df.columns = ["Poäng"]
qual_score_df["Antal Rätt"] = pd.DataFrame(list(qual_df.apply(lambda x: [int(x["Correct"] in qual_df[c].values) for c in qual_df.columns[:4]], axis=1).values), columns=qual_df.columns[:4]).sum()
qual_score_df.sort_values("Poäng", ascending=False)

Antal poäng från lag kvalificerade till slutspel, utifrån gruppställning efter 25 matcher


,Poäng,Antal Rätt
Irannejad-Armin,31.0,11
Sellberg,30.0,11
Martin-Hunhammar,26.0,10
Filip,23.0,9
